In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from ROOT import geo2d
pygeo = geo2d.PyDraw()
from ROOT import larocv
import numpy as np
import cv2

import ROOT, sys, os
from ROOT import std
from larcv import larcv
if len(sys.argv) < 2:
    print 'Usage: python',sys.argv[0],'CONFIG_FILE [LARCV_FILE1 LARCV_FILE2 ...]'
    sys.exit(1)
proc = larcv.ProcessDriver('ProcessDriver')
sys.argv=[""]*4
BASEDIR="/Users/dayajun/larcv/app/LArOpenCVHandle/"
sys.argv[1]=os.path.join(BASEDIR,"cfg/fullchain_larbysimageana.cfg")
sys.argv[2]=os.path.join(BASEDIR,"cfg/mac/fcn.root")
sys.argv[3]="~/fcn_out.root"
proc.configure(sys.argv[1])

if len(sys.argv) > 1:
   flist=ROOT.std.vector('std::string')()
   for x in xrange(len(sys.argv)-3):
      flist.push_back(sys.argv[x+3])
   proc.override_input_file(flist)
ana_id = proc.process_id("LArbysImageAna")
extract_id = proc.process_id("LArbysImageExtract")
larbysimg_ana = proc.process_ptr(ana_id)
larbysimg_ana.SetInputLArbysFile(sys.argv[2])
larbysimg_extract = proc.process_ptr(extract_id)
proc.initialize()

In [ ]:
type2col=["orange","yellow","green","cyan","orange","magenta"]
type2vtx=["unknown","time","wire","shower","end-of-track","magenta"]

ends=[]
pxs=[]
lastentry=0
_vtx_ana = larocv.VertexAnalysis()
for iy in xrange(5):
    for entry in xrange(lastentry,1000):
        lastentry+=1
        if not proc.process_entry(entry): continue
        break

    mcvtx = larbysimg_ana.TrueVertex()
    print "Looking @ Entry, ",iy
    xmin=ymin=99999
    xmax=ymax=-99999
    timg_v=[None]*3
    simg_v=[None]*3
    for plane in xrange(3):
        xmin_=mcvtx.vtx2d_v[plane].pt.x-256
        xmax_=mcvtx.vtx2d_v[plane].pt.x+256
        ymin_=mcvtx.vtx2d_v[plane].pt.y-256
        ymax_=mcvtx.vtx2d_v[plane].pt.y+256

        if xmin>xmin_:xmin=xmin_
        if xmax<xmax_:xmax=xmax_
        if ymin>ymin_:ymin=ymin_
        if ymax<ymax_:ymax=ymax_
        _vtx_ana.ResetPlaneInfo(larbysimg_extract.ADCMeta(plane))
    
    for plane in xrange(3):
        timg=pygeo.image(larbysimg_extract.TrackImage(plane))
        simg=pygeo.image(larbysimg_extract.ShowerImage(plane))
        timg=np.where(timg>10,255,0.0)
        timg_v[plane]=timg.copy().astype(np.uint8)

        simg=np.where(simg>10,150,0.0)
        simg_v[plane]=simg.copy().astype(np.uint8)
    
    if larbysimg_ana.Verticies().empty():
        continue

    for ix,vertex in enumerate(larbysimg_ana.Verticies()):
        fig, axarr = plt.subplots(3,1 , sharex=True,figsize=(20,20))
        good=True
        print 'Event ', iy 
        print "ON Vertex ID ",ix
        pcluster_vv=ROOT.std.vector(ROOT.std.vector(larocv.data.ParticleCluster))()
        for plane in xrange(3):
            pcluster_v=ROOT.std.vector(larocv.data.ParticleCluster)()
            pcluster_vv.push_back(larbysimg_ana.Particles(ix,plane))
        TwoPlanesTwoVertex=_vtx_ana.RequireParticleCount(pcluster_vv,2,2)
        print "2 planes Vertex? ",TwoPlanesTwoVertex
        if TwoPlanesTwoVertex==False:
            good=False
        match_vv = _vtx_ana.MatchClusters(pcluster_vv,larbysimg_extract.ADCImages(),0.5,2,2)
        if match_vv.empty():
            print "No match for vertex of type ",vertex.type,False
            good=False
        print "Found ",match_vv.size(),"matches."
        for iy,match_v in enumerate(match_vv):
            match_arr=np.array(match_v)
                
        for plane in xrange(3):
            ax=axarr[plane]
            ax.imshow(timg_v[plane]+simg_v[plane],interpolation="None",vmin=0,vmax=255,cmap="Greys")
            ax.set_ylim(ymin,ymax)
            ax.set_xlim(xmin,xmax)
            ax.set_aspect(1/3.)
            ax.grid()
            
            for particle in larbysimg_ana.Particles(ix,plane):
                ctor=np.array([[pt.x,pt.y] for pt in particle._ctor])
                color = type2col[particle.type]
                ax.plot(ctor[:,0],ctor[:,1],lw=1,color=color)
            for track in larbysimg_ana.Tracks(ix,plane):
                endpt=track.end_pt()
                if endpt.x ==0 and endpt.y ==0: continue
                ax.plot(endpt.x,endpt.y,'o',color='red',markersize=5)
            vtx2d=vertex.vtx2d_v[plane]
            color_=type2col[vertex.type]
            alpha_=0.70
            if good==False:
                color_="Grey"
                alpha_=0.5
            ax.plot(mcvtx.vtx2d_v[plane].pt.x, mcvtx.vtx2d_v[plane].pt.y, 'o', markersize=15, alpha=alpha_)    
            ax.plot(vtx2d.pt.x,vtx2d.pt.y,'*',color=color_,markersize=20,alpha=alpha_)
            ax.grid()
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.show()
